In [2]:
!pip install yolov5


ERROR: Could not find a version that satisfies the requirement yolov5 (from versions: none)
ERROR: No matching distribution found for yolov5


In [3]:
pip install --upgrade ultralytics


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pillow>=7.1.2 in c:\users\amal p\appdata\roaming\python\python39\site-packages (from ultralytics) (10.1.0)


In [4]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
import warnings
from tracker import*

C:\Users\AMAL P\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\AMAL P\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
C:\Users\AMAL P\AppData\Roaming\Python\Python39\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


ERROR setting UTF-8 encoding: 'OutStream' object has no attribute 'buffer'


In [89]:
model=YOLO('yolov8s.pt')




In [92]:
area1=[(312,388),(289,390),(474,469),(497,462)]

area2=[(279,392),(250,397),(423,477),(454,469)]
#area1=[(312,388),(497,390),(474,469),(497,462)]


#area2=[(279,392),(454,450),(423,477),(454,469)]
def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)
        


In [94]:

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

cap=cv2.VideoCapture('http://192.168.147.201:8080/video')
if not cap.isOpened():
    print("Error: Could not open video source.")
    
    
my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n") 
count=0
tracker=Tracker()
people_entering={}
entering=set()
people_leaving={}
leaving=set()
while True:
    
    # Ignore all warnings
    warnings.filterwarnings("ignore")

    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 2 != 0:
        continue
    frame=cv2.resize(frame,(1020,500))
#    frame=cv2.flip(frame,1)
    results=model.predict(frame)
    #print("---------------------")
    #print(results)
    #print("---------------------")
    
 #   print(results)
    a = results[0].boxes

# Extracting relevant information
    data = {
        "x1": a.xyxy[:, 0].numpy(),
        "y1": a.xyxy[:, 1].numpy(),
        "x2": a.xyxy[:, 2].numpy(),
        "y2": a.xyxy[:, 3].numpy(),
        "confidence": a.conf.numpy(),
        "class": a.cls.numpy()
    }

    # Create a DataFrame
    px = pd.DataFrame(data).astype("float")

    # Print the resulting DataFrame
    #print(px)

    list=[]
    #print("-----------------------")         
    for index,row in px.iterrows():
#        print(row)
 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        #print(row)
        #print(int(row[1]))
        #print(f"Debug: d={d}")
        #print(class_list)
        #print(len(class_list))
        c = class_list[d]
       # print(len(class_list))
       
            
        if 'person' in c:
            list.append([x1,y1,x2,y2])
    bbox_id=tracker.update(list)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        
        #people entering
        results=cv2.pointPolygonTest(np.array(area2,np.int32),((x4,y4)),False)
        if results>=0:
            people_entering[id]=(x4,y4)
            cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 0, 255), 2)
        if id in people_entering:
            results1=cv2.pointPolygonTest(np.array(area1,np.int32),((x4,y4)),False)
            if results1>=0:
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
                cv2.circle(frame,(x4,y4),5,(255,0,255),-1)
                cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                entering.add(id)
                
        #people Leaving
        results2=cv2.pointPolygonTest(np.array(area1,np.int32),((x4,y4)),False)
        if results2>=0:
            people_leaving[id]=(x4,y4)
            cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
        if id in people_leaving:
            results3=cv2.pointPolygonTest(np.array(area2,np.int32),((x4,y4)),False)
            if results3>=0:
                cv2.rectangle(frame, (x3, y3), (x4, y4), (255,0 , 255), 2)
                cv2.circle(frame,(x4,y4),5,(255,0,255),-1)
                cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                leaving.add(id)

        
    cv2.polylines(frame,[np.array(area1,np.int32)],True,(255,0,0),2)
    cv2.putText(frame,str('1'),(504,471),cv2.FONT_HERSHEY_COMPLEX,(0.5),(0,0,0),1)

    cv2.polylines(frame,[np.array(area2,np.int32)],True,(255,0,0),2)
    cv2.putText(frame,str('2'),(466,485),cv2.FONT_HERSHEY_COMPLEX,(0.5),(0,0,0),1)
    
    cv2.putText(frame,str(len(entering)-len(leaving)),(60,80),cv2.FONT_HERSHEY_COMPLEX,(0.7),(0,0,255),2)


    cv2.imshow("RGB", frame)
    if cv2.waitKey(1)&0xFF==27:        
        break
        
cap.release()
cv2.destroyAllWindows()


0: 320x640 5 persons, 167.4ms
Speed: 0.0ms preprocess, 167.4ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
[440, 244]
[440, 244]

0: 320x640 5 persons, 1 bed, 170.9ms
Speed: 0.0ms preprocess, 170.9ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 persons, 1 bed, 168.4ms
Speed: 0.0ms preprocess, 168.4ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 2 beds, 154.8ms
Speed: 5.0ms preprocess, 154.8ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 5 persons, 156.0ms
Speed: 3.1ms preprocess, 156.0ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 persons, 156.9ms
Speed: 13.3ms preprocess, 156.9ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 166.4ms
Speed: 3.4ms preprocess, 166.4ms inference, 7.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 1 bed, 164.6ms
Speed: 2.2ms p